In [50]:
import pandas as pd
import numpy as np
path='C:\\Users\\zhangchen20\\Desktop\\time_test.xlsx'
file_practice=pd.read_excel(path)

In [2]:
file_practice.keys()

Index(['app_ord_dp_overseas_coupon_quantity.product_id',
       'app_ord_dp_overseas_coupon_quantity.platform',
       'app_ord_dp_overseas_coupon_quantity.city_id',
       'app_ord_dp_overseas_coupon_quantity.city_name',
       'app_ord_dp_overseas_coupon_quantity.couponsource',
       'app_ord_dp_overseas_coupon_quantity.coupontitle',
       'app_ord_dp_overseas_coupon_quantity.region',
       'app_ord_dp_overseas_coupon_quantity.getuser',
       'app_ord_dp_overseas_coupon_quantity.useuser',
       'app_ord_dp_overseas_coupon_quantity.getnum',
       'app_ord_dp_overseas_coupon_quantity.usenum',
       'app_ord_dp_overseas_coupon_quantity.partition_date'],
      dtype='object')

In [3]:
#pd.to_datetime(file_practice['app_ord_dp_overseas_coupon_quantity.partition_date'])
def date_split(date):
    y,m,d=date.split('-')
    return '%s年-%s月'%(y,m)
Year_Mon=file_practice['app_ord_dp_overseas_coupon_quantity.partition_date'].map(date_split)
file_practice['Year-Mon']=Year_Mon
#file_practice.head(2)

In [51]:
#先设置datetimeindex， 再重采样
file_practice['date']=pd.to_datetime(file_practice['app_ord_dp_overseas_coupon_quantity.partition_date'])

#file_practice['date'].resample('y',how='sum') 这个可以跑一下，看下为什么会有
file2=file_practice
file2=file2.set_index(file2.date).resample('w').sum()


In [55]:
# 根据ID来分组，然后对每组进行重采样

'''
用下面这个是没有用的
def groupbyID_resemple_byweek(group):
    group.resample('w').count()
'''
file_practice.set_index(file_practice.date,inplace=True) #经过测试，折一行是必须的，只有有了DatetimeIndex才可以用dataframe.resample
file_groupbyID_resemple_bywee=file_practice.groupby('app_ord_dp_overseas_coupon_quantity.product_id').resample('w').sum()